In [1]:
import json

In [3]:
sourceFile = 'outputs/transcripts/sjV7NNwm1GU.json'
with open(sourceFile, 'r') as f:
    data = json.load(f)

text_content = ' '.join([item['text'] for item in data])

with open('output.txt', 'w') as f:
    f.write(text_content)

In [ ]:
#"model": "ifioravanti/mistral-grammar-checker",
#"model": "phi4:latest",

In [30]:
import httpx
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

OLLAMA_ENDPOINT = "http://localhost:11434/api/generate"

def fix_punctuation(text):
    prompt = f"""Fix all punctuation in this text, but preserve:
        1. Original wording
        2. Capitalization
        3. New line characters

    Return only the corrected text:

    {text}
    """

    response = httpx.post(
        OLLAMA_ENDPOINT,
        json={
            "model": "llama3.3:latest",
            "prompt": prompt,
            "stream": False,
            "options": {
                "temperature": 0.1,
                "num_ctx": 4096
            }
        },
        headers={"Content-Type": "application/json"},
        timeout=500
    )

    if response.status_code != 200:
        logger.error(f"Error {response.status_code}: {response.text}")
        return None

    return response.json()["response"].strip()

# Read the transcript file
with open("output.txt", "r") as file:
    transcript = file.read()

# Fix punctuation
corrected_transcript = fix_punctuation(transcript)

# Save the corrected transcript
if corrected_transcript is not None:
    with open("corrected_transcript.txt", "w") as file:
        file.write(corrected_transcript)
    logger.info("Punctuation correction completed successfully.")
else:
    logger.error("Failed to correct punctuation.")

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:__main__:Punctuation correction completed successfully.
